Getting UseCase 3

In [ ]:
cd /home/karen/Documents/GitHub/Muscle_wasting

In [ ]:
import mirna_scoring.mirna_impact as mis
import mirna_scoring.mirna_influence_plots as mi_plot
from network.network_processing import load_graph

In [ ]:
network = load_graph(f"network/Networks_pkl/complete_n_tf_mirnas__UseCase5_cutoff_0.95.pkl")
session_name = 'UseCase_5'

In [ ]:
pathway_keywords=['ATP', 'APOPTOSIS', 'MUSCLE', "MYO", "INFLAMMATION","CYTOKIN", "OXIDATION", "AUTOPHAG", "MACROPHAG", "MYTOPHAG", "MITO", "HGF", "SATELLITE"]
steps=10
sample_size=10

In [ ]:
mirna_network=mis.mirna_network(network)
evaluation = mis.mirna_evaluation(mirna_network, session_name)

In [ ]:
len(evaluation.get_all_mirnas())

In [ ]:
evaluation_df = evaluation.score(steps=steps, sample_size=sample_size, dds_threshold=2, pathway_keywords=pathway_keywords)

In [ ]:
aa=evaluation.select_mirnas()
dict(zip(list(aa),[list(aa).count(i) for i in list(aa)]))

In [ ]:
n_clusters = len(mirna_network.miR_nodes)//3
evaluation.cluster_mirnas(n_clusters=n_clusters)

In [ ]:
dist_df = mirna_network.get_mirnas_similarity()
mi_plot.plot_mirnas_similarirty(dist_df)

In [ ]:
evaluation.create_report_clusters()

In [ ]:
genes = mirna_network.network.nodes
genes = list(set(genes))

In [ ]:
len(genes)

In [ ]:
import decoupler as dc
from mirkitten.plot_GSEA_ORA import plot_ora_results
enriched = dc.get_ora_df(
            df=genes,
            net=evaluation.msigdb,
            source='geneset',
            target='genesymbol'
        )
pathway_df = enriched[enriched['FDR p-value'] < 0.1]
pathway_df.index = pathway_df["Term"]
#pathway_df.set_index("Term", inplace=True)  # Set "Term" as index
enriched_pathways = pathway_df['Combined score']
plot_ora_results(pathway_df, top_n=10, figsize=(12, 8), scale_odds_ratio=20, 
                     fontsize_title=16, fontsize_subtitle=14, fontsize_text=12,)

In [ ]:
mirnas_selected = list(set(aa))
mirna_clusters = evaluation.clustered_mirnas

In [ ]:
import umap
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
selected_dist_df = dist_df.loc[mirnas_selected, mirnas_selected]
# Compute UMAP embedding
reducer = umap.UMAP(metric="precomputed", random_state=42)
embedding = reducer.fit_transform(selected_dist_df)

# Create a dataframe with UMAP results and cluster labels

plot_df = pd.DataFrame(embedding, columns=['UMAP1', 'UMAP2'], index=selected_dist_df.index)
plot_df['Cluster'] = pd.Series({mir: cluster for cluster, mirs in mirna_clusters.items() for mir in mirs}, name="Cluster")

# Plot the clusters
plt.figure(figsize=(10, 7))
sns.scatterplot(x='UMAP1', y='UMAP2', hue='Cluster', data=plot_df, palette='tab10', alpha=0.7)

# Add labels for each miRNA
for i, txt in enumerate(plot_df.index):
    plt.annotate(txt, (plot_df['UMAP1'][i], plot_df['UMAP2'][i]), fontsize=8, alpha=0.7)

plt.title("UMAP Visualization of Spectral Clustering")
plt.legend(title="Cluster", bbox_to_anchor=(1, 1))
plt.show()


In [ ]:
mi_plot.plot_mirnas_similarirty(selected_dist_df)

In [ ]:
sarcopenia_pathways = [
        "mTOR","AKT","PI3K","FoxO","MAPK","JAK-STAT","NF-kB","AMPK","PGC-1alpha",
        "TGF-beta","Myostatin","Wnt","Notch","Hedgehog","IGF-1","Autophagy","Ubiquitin-proteasome",
        "Proteostasis","Oxidative_stress","Inflammation","Mitochondrial_dysfunction","Apoptosis",
        "ER_stress","Unfolded_protein_response","Calcium_signaling","Excitation-contraction_coupling",
        "Hypoxia","Angiogenesis","HIF-1alpha","Glucose_metabolism","Fatty_acid_oxidation",
        "Insulin_resistance","Sirtuins","SIRT1","SIRT3","Follistatin","Acetylation","Histone_deacetylation",
        "Senescence","Fibrosis","Satellite_cells","Muscle_regeneration","Stem_cell_exhaustion",
        "Circadian_rhythm","Glycolysis","TCA_cycle","ROS","NO_signaling","GDF11","NAD+_metabolism",
        "DNA_damage","Protein_synthesis","Proteolysis","Chaperones","Heat_shock_proteins","Lipid_metabolism",
        "Cytokines","TNF-alpha","IL-6","IL-10","Fibroblast_growth_factor","Exercise_response","Neuromuscular_junction",
        "Denervation","Neuropathy","Acetylcholine","Synaptic_plasticity","Epigenetics","DNA_methylation",
        "MicroRNAs","Long_noncoding_RNAs","Exosomes","Extracellular_vesicles","Glycogen_metabolism",
        "Serotonin","Dopamine","Adrenergic_signaling","Thyroid_hormone","Glucocorticoids","Androgens",
        "Estrogens","Testosterone","Vitamin_D","Polyamine_metabolism","Nutrient_sensing","Ketone_bodies",
        "HMB","Leucine","Branched-chain_amino_acids","TORC1","TORC2","E3_ligases","Atrogin-1","MuRF1",
        "Fibronectin","Integrins","Focal_adhesion","YAP/TAZ","Mechanical_stress","Stretch_signaling"
]
sarcopenia_pathways = list(map(str.upper,sarcopenia_pathways))
sarcopenia_pathways[:5]

In [ ]:

short_msigdb = evaluation.msigdb[evaluation.msigdb['geneset'].str.contains('|'.join(sarcopenia_pathways), na=False)]
#evaluation.msigdb


In [ ]:
short_msigdb.head()

In [ ]:

db = evaluation.msigdb
short_msigdb_by_collection = {key: db[db["collection"] == key] for key in db["collection"].unique()}


In [ ]:
short_msigdb_by_collection.keys()

In [ ]:
import decoupler as dc
from mirkitten.plot_GSEA_ORA import plot_ora_results

for collection, short_msigdb_collection in short_msigdb_by_collection.items():
    
    enriched = dc.get_ora_df(
                df=genes,
                net=pd.DataFrame(short_msigdb_collection),
                source='geneset',
                target='genesymbol'
            )
    pathway_df = enriched[enriched['FDR p-value'] < 0.1]
    pathway_df.index = pathway_df["Term"]
    #pathway_df.set_index("Term", inplace=True)  # Set "Term" as index
    enriched_pathways = pathway_df['Combined score']
    title=f"Whole network top 10 of {collection}"
    fig, ax = plot_ora_results(pathway_df, top_n=10, figsize=(12, 8), scale_odds_ratio=20, 
                         fontsize_title=16, fontsize_subtitle=14, fontsize_text=12, title=title)
    pathway_df.to_csv(f"mirna_scoring/results/{session_name}/{title}.csv")
    image_file = f"mirna_scoring/results/{session_name}/All_network_{collection}.png"
    fig.savefig(image_file, dpi=300, bbox_inches='tight')

In [ ]:
full_msigdb = evaluation.msigdb
evaluation.msigdb = short_msigdb

In [ ]:
evaluation.create_report_clusters(save_directory=f"mirna_scoring/results/{session_name}/only_sarcopenia_pathways")

In [ ]:

db = evaluation.msigdb
short_msigdb_by_collection = {key: db[db["collection"] == key] for key in db["collection"].unique()}


In [ ]:
for collection, short_msigdb_collection in short_msigdb_by_collection.items():
    
    enriched = dc.get_ora_df(
                df=genes,
                net=pd.DataFrame(short_msigdb_collection),
                source='geneset',
                target='genesymbol'
            )
    pathway_df = enriched[enriched['FDR p-value'] < 0.1]
    pathway_df.index = pathway_df["Term"]
    #pathway_df.set_index("Term", inplace=True)  # Set "Term" as index
    enriched_pathways = pathway_df['Combined score']
    title=f"Whole network top 10 of {collection} - Only relevant pathways"
    fig, ax = plot_ora_results(pathway_df, top_n=10, figsize=(12, 8), scale_odds_ratio=20, 
                         fontsize_title=16, fontsize_subtitle=14, fontsize_text=12, title=title)
    pathway_df.to_csv(f"mirna_scoring/results/{session_name}/{title}.png")
    image_file = f"mirna_scoring/results/{session_name}/only_sarcopenia_pathways/All_network_{collection}.png"
    fig.savefig(image_file, dpi=300, bbox_inches='tight')